In [ ]:
# Contraception Dataset


### Import all modules.


In [ ]:
import numpy as np
from pandas import read_csv
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint as sp_randint


In [ ]:
### Prepare the dataset.


In [ ]:
# Read the dataset.
dataset = read_csv('datasets/contraception.csv', engine='python')
# Get x and y.
X, y = dataset.iloc[:, :-1].values, dataset.iloc[:, -1].values
# Split to training and test pairs.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
### Create the Bagging Ensemble.


In [16]:
# Specify parameters and distributions to sample from.
param_dist = {"max_depth": sp_randint(4, 30),
              "max_features": sp_randint(1, 9),
              "min_samples_split": sp_randint(2, 11),
              "criterion": ["gini", "entropy"],
              'n_estimators': sp_randint(10, 80)}
# 
clf = RandomizedSearchCV(RandomForestClassifier(random_state=0), param_dist, cv=10, n_jobs=-1, verbose=5)
clf.fit(X_train, y_train)
# Create a bagging DecisionTree classifier.
bagging = BaggingClassifier(clf, n_estimators=100, n_jobs=-1, random_state=0)
model = bagging.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

c:\users\manos\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


KeyboardInterrupt: 